In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw4.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 4: Logistic regression, hyperparameter optimization 
### Associated lectures: [Lectures 7, 8](https://ubc-cs.github.io/cpsc330-2025W2/#lecture-schedule-tentative)

**Due date: See [deliverable due dates](https://ubc-cs.github.io/cpsc330-2025W2/#deliverable-due-dates-tentative)**.

## Imports 

In [2]:
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["font.size"] = 16

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV, 
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier

In [3]:
import hashlib


def my_sha1(text):
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W2/syllabus#use-of-generative-ai-in-the-course)** before starting the homework assignment. 
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W2/docs/homework-instructions) for detailed guidance on completing and submitting assignments. 
   
</div>

_Points:_ 4

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1: implementing `DummyClassifier`
<hr>
rubric={autograde}

In this course (unlike CPSC 340) you will generally **not** be asked to implement machine learning algorihtms (like logistic regression) from scratch. However, this exercise is an exception: you will implement the simplest possible classifier, `DummyClassifier`.
 
As a reminder, `DummyClassifier` is meant as a baseline and is generally a simple "model" you could "fit" to a dataset. All it does is predict the most popular class in the training set. So if there are more 0s than 1s it predicts 0 every time, and if there are more 1s than 0s it predicts 1 every time. For `predict_proba` it looks at the frequencies in the training set, so if you have 30% 0's and 70% 1's it predicts `[0.3 0.7]` every time. Thus, `fit` only looks at `y` (not `X`).

Below you will find starter code for a class called `MyDummyClassifier`, which has methods `fit()`, `predict()`, `predict_proba()` and `score()`. Your task is to fill in those four functions. To get you started, I have given you a `return` statement in each case that returns the correct data type: 
- `fit` returns nothing
- `predict` returns an array whose size is the number of examples
- `predict_proba` returns an array whose size is the number of examples x 2, and
- `score` returns a float

The next code block has some tests you can use to assess whether your code is working. 

I suggest starting with `fit` and `predict`, and making sure those are working before moving on to `predict_proba`. For `predict_proba`, you should return the frequency of each class in the training data. **Your `score` function should call your `predict` function**. Again, you can compare with `DummyClassifier` using the code below.

`sklearn`'s `DummyClassifier` works when you have more than two classes, and also works if the target values are encoded differently, for example as "cat", "dog", "eagle", etc. However, for the sake of simplifying this question, we will consider **binary classification** only. Furthermore, we will assume that these classes are encoded as 0 and 1. In other words, you can safely assume that the variable y contains only 0's and 1's

<div class="alert alert-warning">

Solution_1
    
</div>

In [4]:
class MyDummyClassifier:
    """
    A baseline classifier that predicts the most common class.
    The predicted probabilities come from the relative frequencies
    of the classes in the training data.

    This implementation only works when y only contains 0's and 1's.
    """

    def fit(self, X, y):
        y = np.asarray(y).astype(int)
        counts = np.bincount(y, minlength=2)
        self.class_prior_ = counts / counts.sum()
        self.most_common_ = int(np.argmax(counts))  # ties -> 0
        return self

    def predict(self, X):
        n = X.shape[0]
        return np.full(n, self.most_common_, dtype=int)

    def predict_proba(self, X):
        n = X.shape[0]
        return np.tile(self.class_prior_, (n, 1))

    def score(self, X, y):
        y = np.asarray(y).astype(int)
        return np.mean(self.predict(X) == y)


Below are some tests for `predict` using randomly generated data. You may want to run the cell a few times to make sure you explore the different cases (or automate this with a loop or random seeds).

In [5]:
# For testing, generate random data
n_train = 101
n_valid = 21
d = 5
X_train_dummy = np.random.randn(n_train, d)
X_valid_dummy = np.random.randn(n_valid, d)
y_train_dummy = np.random.randint(2, size=n_train)
y_valid_dummy = np.random.randint(2, size=n_valid)

my_dc = MyDummyClassifier()
sk_dc = DummyClassifier(strategy="prior")

my_dc.fit(X_train_dummy, y_train_dummy)
sk_dc.fit(X_train_dummy, y_train_dummy)

assert np.array_equal(my_dc.predict(X_train_dummy), sk_dc.predict(X_train_dummy))
assert np.array_equal(my_dc.predict(X_valid_dummy), sk_dc.predict(X_valid_dummy))

Below are some tests for `predict_proba`.

In [6]:
assert np.allclose(
    my_dc.predict_proba(X_train_dummy), sk_dc.predict_proba(X_train_dummy)
)
assert np.allclose(
    my_dc.predict_proba(X_valid_dummy), sk_dc.predict_proba(X_valid_dummy)
)

Below are some tests for `score`.

_Points:_ 10

In [7]:
assert np.isclose(
    my_dc.score(X_train_dummy, y_train_dummy), sk_dc.score(X_train_dummy, y_train_dummy)
)
assert np.isclose(
    my_dc.score(X_valid_dummy, y_valid_dummy), sk_dc.score(X_valid_dummy, y_valid_dummy)
)

In [8]:
grader.check("q1")

q1 results:
    q1 - 1 result:
        ❌ Test case failed
        Trying:
            X_train_dummy_tests = np.random.randn(51, 5)
        Expecting nothing
        ok
        Trying:
            y_train_dummy_tests = np.append(np.ones(26), np.zeros(25))
        Expecting nothing
        ok
        Trying:
            X_valid_dummy_tests = np.random.randn(11, 5)
        Expecting nothing
        ok
        Trying:
            y_valid_dummy_tests = np.append(np.zeros(7), np.ones(4))
        Expecting nothing
        ok
        Trying:
            test_dc = MyDummyClassifier()
        Expecting nothing
        ok
        Trying:
            test_dc.fit(X_train_dummy_tests, y_train_dummy_tests)
        Expecting nothing
        **********************************************************************
        Line 6, in q1 0
        Failed example:
            test_dc.fit(X_train_dummy_tests, y_train_dummy_tests)
        Expected nothing
        Got:
            <__main__.MyDummyClassifier object at 0x0000016CBACB9810>
        Trying:
            assert np.array_equal(test_dc.predict(X_train_dummy_tests), np.ones(51)), 'Tests for predict failed'
        Expecting nothing
        ok
        Trying:
            assert np.array_equal(test_dc.predict(X_valid_dummy_tests), np.ones(11)), 'Tests for predict failed'
        Expecting nothing
        ok
        Trying:
            predict_test = test_dc.predict_proba(X_train_dummy_tests)
        Expecting nothing
        ok
        Trying:
            assert np.all(predict_test == predict_test[0]), 'Tests for predict_proba failed'
        Expecting nothing
        ok
        Trying:
            assert np.allclose(predict_test[0], np.mean(test_dc.predict_proba(X_valid_dummy_tests), axis=0), [0.49, 0.51]), 'Tests for predict_proba failed'
        Expecting nothing
        ok
        Trying:
            assert np.isclose(test_dc.score(X_train_dummy_tests, y_train_dummy_tests), 0.51, atol=0.01), 'Tests for score failed'
        Expecting nothing
        ok
        Trying:
            assert np.isclose(test_dc.score(X_valid_dummy_tests, y_valid_dummy_tests), 0.364, atol=0.01), 'Tests for score failed'
        Expecting nothing
        ok

    q1 - 2 result:
        ❌ Test case failed
        Trying:
            X_train_dummy_tests = np.random.randn(35, 5)
        Expecting nothing
        ok
        Trying:
            y_train_dummy_tests = np.append(np.ones(10), np.zeros(25))
        Expecting nothing
        ok
        Trying:
            X_valid_dummy_tests = np.random.randn(7, 5)
        Expecting nothing
        ok
        Trying:
            y_valid_dummy_tests = np.append(np.zeros(1), np.ones(6))
        Expecting nothing
        ok
        Trying:
            test_dc = MyDummyClassifier()
        Expecting nothing
        ok
        Trying:
            test_dc.fit(X_train_dummy_tests, y_train_dummy_tests)
        Expecting nothing
        **********************************************************************
        Line 6, in q1 1
        Failed example:
            test_dc.fit(X_train_dummy_tests, y_train_dummy_tests)
        Expected nothing
        Got:
            <__main__.MyDummyClassifier object at 0x0000016CBACB9E50>
        Trying:
            assert np.all(test_dc.predict(X_train_dummy_tests) == 0), 'Tests for predict failed'
        Expecting nothing
        ok
        Trying:
            assert np.all(test_dc.predict(X_valid_dummy_tests) == 0), 'Tests for predict failed'
        Expecting nothing
        ok
        Trying:
            predict_test = test_dc.predict_proba(X_train_dummy_tests)
        Expecting nothing
        ok
        Trying:
            assert np.all(predict_test == predict_test[0]), 'Tests for predict_proba failed'
        Expecting nothing
        ok
        Trying:
            assert np.allclose(predict_test[0], np.mean(test_dc.predict_proba(X_valid_dummy_tests), axis=0), [0.714, 0.286]), 'Tests for predict_proba failed'
        Expecting nothing
        

<br><br><br><br>

## Exercise 2: Trump Tweets
<hr>

For the rest of this assignment we'll be working with a [dataset of Donald Trump's tweets](https://www.kaggle.com/austinreese/trump-tweets) as of June 2020. You should start by downloading the dataset. Unzip it and move the file `realdonaldtrump.csv` under the data directory in this folder. As usual, please do not submit the dataset when you submit the assignment. 

In [9]:
tweets_df = pd.read_csv("data/realdonaldtrump.csv", index_col=0)
tweets_df.head()

,link,content,date,retweets,favorites,mentions,hashtags
id,,,,,,,
1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


In [10]:
tweets_df.shape

(43352, 7)

We will be trying to predict whether a tweet will go "viral", defined as having more than 10,000 retweets:

In [11]:
y = tweets_df["retweets"] > 10_000
y.value_counts()

retweets
False    31939
True     11413
Name: count, dtype: int64

To make predictions, we'll be using only the content (text) of the tweet. 

In [12]:
X = tweets_df["content"]

For the purpose of this assignment, you can ignore all the other columns in the original dataset.

<br><br>

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Ordering the steps
rubric={points}

Let's start by building a model using `CountVectorizer` and `LogisticRegression`. The code required to do this has been provided below, but in the wrong order. 

**Your tasks:**
1. Rearrange the lines of code to correctly fit the model and compute the cross-validation score. 
2. Add a short comment above each step to describe what the code is doing.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 3

In [13]:
# YOUR COMMENT HERE
countvec = CountVectorizer(stop_words="english")

# YOUR COMMENT HERE
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.6, random_state=123
)

# YOUR COMMENT HERE
cv_score = cross_val_score(pipe, X_train, y_train).mean()

# YOUR COMMENT HERE
pipe = make_pipeline(countvec, lr)

# YOUR COMMENT HERE
lr = LogisticRegression(max_iter=1000, random_state=123)

NameError: name 'pipe' is not defined

In [14]:
# Create the text vectorizer (bag-of-words).
countvec = CountVectorizer(stop_words="english")

# Create the classifier.
lr = LogisticRegression(max_iter=1000, random_state=123)

# Build the pipeline: vectorize text -> fit logistic regression.
pipe = make_pipeline(countvec, lr)

# Split into train/test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=123)

# Compute mean cross-validation accuracy on the training set.
cv_score = cross_val_score(pipe, X_train, y_train).mean()
...

Ellipsis

<!-- END QUESTION -->

<br><br>

### 2.2 Baseline
rubric={autograde}

**Your tasks:**

1. Run 5-fold cross-validation with `sklearn`'s `DummyClassifier` on this dataset. Store the mean cross-validation score in a variable called `dummy_cv_score` given below.

> You should be able to use `MyDummyClassifier` above. But sklearn will likely complain if you try to pass it to `cross_val_score` or `cross_validate`. 

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 2

In [15]:
dummy = DummyClassifier(strategy="prior", random_state=123)
dummy_cv_score = cross_val_score(dummy, X_train, y_train, cv=5).mean()
dummy_cv_score

np.float64(0.7423875432525952)

In [16]:
grader.check("q2.2")

q2.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Discussion
rubric={points}

**Your tasks:**

1. Compare the `DummyClassifier` score to what you got from logistic regression above. Does logistic regression seem to be doing anything useful?
2. Above we averaged the scores from the 5 folds of cross-validation of logistic regression. Print out the 5 individual scores. Reminder: `sklearn` calls them `"test_score"` but they are really (cross-)validation scores. 
3. Are the 5 scores close to each other or spread far apart? (This is a bit subjective, answer to the best of your ability.) 

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 3

Logistic regression’s mean CV accuracy is higher than the DummyClassifier’s, so it’s learning real signal from the tweet text instead of just guessing the most common label. The fold scores are fairly consistent (no huge swings), which suggests the estimate is reasonably stable across splits.

In [17]:
...

Ellipsis

In [18]:
...

Ellipsis

In [19]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

### 2.4 Probability scores
rubric={autograde}

Here we train a logistic regression classifier on the entire training set: 

In [20]:
pipe_lr = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression(max_iter=1000, random_state=123),
)
pipe_lr.fit(X_train, y_train);

**Your tasks:**

1. Using this model, find the tweet in the **test set** with the highest predicted probability of being viral. Store the tweet and the associated probability in the variables `tweet` and `prob`, respectively. 

> Reminder: you are free to reuse/adapt code from lecture. Please add in a small attribution, e.g. "From Lecture 7".

<div class="alert alert-warning">

Solution_2.4
    
</div>

_Points:_ 4

In [21]:
 
probs_pos = pipe_lr.predict_proba(X_test)[:, 1]
best_i = int(np.argmax(probs_pos))
tweet = X_test.iloc[best_i] if hasattr(X_test, "iloc") else X_test[best_i]
prob = float(probs_pos[best_i])
tweet, prob

('There was No Collusion with Russia (except by the Democrats). When will this very expensive Witch Hunt Hoax ever end? So bad for our Country. Is the Special Counsel/Justice Department leaking my lawyers letters to the Fake News Media? Should be looking at Dems corruption instead?',
 0.9999999830416225)

In [22]:
grader.check("q2.4")

q2.4 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Coefficients
rubric={points}

We can extract the `CountVectorizer` and `LogisticRegression` objects from the `Pipeline` object as follows:

In [23]:
vec_from_pipe = pipe_lr.named_steps["countvectorizer"]
lr_from_pipe = pipe_lr.named_steps["logisticregression"]

**Your tasks:**

Display the top 5 and bottom 5 words, together with their corresponding coefficients learned by the logistic regression model.

<div class="alert alert-warning">

Solution_2.5
    
</div>

_Points:_ 4

In [24]:
# Get feature names and coefficients
feature_names = vec_from_pipe.get_feature_names_out()
coefs = lr_from_pipe.coef_.ravel()

# Top 5 (most positive) and bottom 5 (most negative)
top_idx = np.argsort(coefs)[-5:][::-1]
bot_idx = np.argsort(coefs)[:5]

top5 = list(zip(feature_names[top_idx], coefs[top_idx]))
bot5 = list(zip(feature_names[bot_idx], coefs[bot_idx]))

top5, bot5...

SyntaxError: invalid syntax (886895501.py, line 12)

<!-- END QUESTION -->

<br><br>

### 2.6 Running a cross-validation fold without sklearn tools 
rubric={autograde}

Sklearn provides a lot of useful tools like `make_pipeline` and `cross_validate`, which are awesome. But with these fancy tools it's also easy to lose track of what is actually happening under the hood. 

**Your tasks:**

1. Compute logistic regression's validation score on the first fold, that is, train on 80% and validate on 20% of the training data (`X_train`) without using sklearn `Pipeline` or `cross_validate` or `cross_val_score`. Store the score of the fold in a variable called `fold_score`. Recall that `cross_validation` in `sklearn` does not shuffle the data by default.    

You should start with the following `CountVectorizer` and `LogisticRegression` objects, as well as `X_train` and `y_train` (which you should further split with `train_test_split` and `shuffle=False`):
> Note: avoid assigning the variable names `X_train`, `y_train`, `X_test`, `y_test` to your folds when you call `train_test_split`. You'll be using those again later in the assignment.

In [ ]:
countvec = CountVectorizer(stop_words="english")
lr = LogisticRegression(max_iter=1000, random_state=123)

> Meta-comment: you might be wondering why we're going into "implementation" here if this course is about _applied_ ML. In CPSC 340, we would go all the way down into `LogisticRegression` and understand how `fit` works, line by line. Here we're not going into that at all, but I still think this type of question (and Exercise 1) is a useful middle ground. I do want you to know what is going on in `Pipeline` and in `cross_validate` even if we don't cover the details of `fit`. To get into logistic regression's `fit` requires a bunch of math; here, we're keeping it more conceptual and avoiding all those prerequisites.

<div class="alert alert-warning">

Solution_2.6
    
</div>

_Points:_ 8

In [ ]:
 
X_tr_fold, X_val_fold, y_tr_fold, y_val_fold = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=False
)
 
Xtr_vec = countvec.fit_transform(X_tr_fold)
Xval_vec = countvec.transform(X_val_fold)
 
lr.fit(Xtr_vec, y_tr_fold)
fold_score = lr.score(Xval_vec, y_val_fold)
 

In [ ]:
fold_score

In [ ]:
grader.check("q2.6")

<br><br><br><br>

## Exercise 3: Hyperparameter optimization
<hr>

<!-- BEGIN QUESTION -->

### 3.1 Optimizing `max_features` of `CountVectorizer`
rubric={points}

The following code varies the `max_features` hyperparameter of `CountVectorizer` and makes a plot (with the x-axis on a log scale) that shows train/cross-validation scores vs. `max_features`. It also prints the results. 

**Your tasks:**
- Based on the plot/output, what value of `max_features` seems best? Briefly explain.

> The code may take a minute or two to run. You can uncomment the `print` statement if you want to see it show the progress.

In [ ]:
train_scores = []
cv_scores = []

max_features = [10, 100, 1000, 10_000, 100_000]

for mf in max_features:
    #     print(mf)
    pipe = make_pipeline(
        CountVectorizer(stop_words="english", max_features=mf),
        LogisticRegression(max_iter=1000, random_state=123),
    )
    cv_results = cross_validate(pipe, X_train, y_train, return_train_score=True)
    train_scores.append(cv_results["train_score"].mean())
    cv_scores.append(cv_results["test_score"].mean())

plt.semilogx(max_features, train_scores, label="train")
plt.semilogx(max_features, cv_scores, label="valid")
plt.legend()
plt.xlabel("max_features");
plt.ylabel("accuracy");

In [ ]:
pd.DataFrame({"max_features": max_features, "train": train_scores, "cv": cv_scores})

<div class="alert alert-warning">

Solution_3.1
    
</div>

_Points:_ 2

I’d pick the max_features value that gives the highest mean CV validation accuracy. If multiple are basically tied, choose the smaller max_features (simpler model) because it’s less likely to overfit and is cheaper to train.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.2 Optimizing `C` of `LogisticRegression`
rubric={points}

The following code varies the `C` hyperparameter of `LogisticRegression` and makes a plot (with the x-axis on a log scale) that shows train/cross-validation scores vs. `C`. 

**Your tasks:**

- Based on the plot, what value of `C` seems best? Briefly explain. 

> The code may take a minute or two to run. You can uncomment the `print` statement if you want to see it show the progress.

In [ ]:
train_scores = []
cv_scores = []

C_vals = 10.0 ** np.arange(-1.5, 2, 0.5)

for C in C_vals:
    #     print(C)
    pipe = make_pipeline(
        CountVectorizer(stop_words="english"),
        LogisticRegression(max_iter=1000, C=C, random_state=123),
    )
    cv_results = cross_validate(pipe, X_train, y_train, return_train_score=True)

    train_scores.append(cv_results["train_score"].mean())
    cv_scores.append(cv_results["test_score"].mean())

plt.semilogx(C_vals, train_scores, label="train")
plt.semilogx(C_vals, cv_scores, label="valid")
plt.legend()
plt.xlabel("C")
plt.ylabel("accuracy");

In [ ]:
pd.DataFrame({"C": C_vals, "train": train_scores, "cv": cv_scores})

<div class="alert alert-warning">

Solution_3.2
    
</div>

_Points:_ 2

I’d pick the C value with the highest mean CV validation accuracy. If several are close, pick the smaller C (stronger regularization) since it usually generalizes better unless the validation scores clearly favor a larger C.

<!-- END QUESTION -->

<br><br>

### 3.3 Jointly optimizing `max_features` of `CountVectorizer` and `C` of `LogisticRegression` using `GridSearchCV`
rubric={autograde}

So far we have been writing loops to try a bunch of different hyperparameter values and pick the one with lowest cross-validation error. This operation is so common that `scikit-learn` has some built-in classes to do it for you. In this and the next exercise, we will explore two such classes:

1. [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 
2. [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

Start with the pipeline `pipe_lr` below.

**Your tasks:**
- Create a `GridSearchCV` object named `grid_search` to jointly optimize `max_features` of `CountVectorizer` and `C` of `LogisticRegression` across all the combinations of values we tried above. 
- What are the best values of `max_features` and `C` according to your grid search? Store them in variables `best_max_features` and `best_C`, respectively.  
- Store the best score returned by the grid search in a variable called `best_score`. 

> The code might be a bit slow here. Setting `n_jobs=-1` should speed it up if you have a multi-core processor.

In [ ]:
pipe_lr = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression(max_iter=1000, random_state=123),
)

<div class="alert alert-warning">

Solution_3.3
    
</div>

_Points:_ 6

In [ ]:
param_grid = {
    "countvectorizer__max_features": [10, 100, 1000, 10_000, 100_000],
    "logisticregression__C": list(10.0 ** np.arange(-1.5, 2, 0.5)),
}

grid_search = GridSearchCV(pipe_lr, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_max_features = grid_search.best_params_["countvectorizer__max_features"]
best_C = grid_search.best_params_["logisticregression__C"]
best_score = grid_search.best_score_

best_max_features, best_C, best_score

 

In [ ]:
best_max_features

In [ ]:
best_C

In [ ]:
best_score

In [ ]:
grader.check("q3.3")

<br><br>

<!-- BEGIN QUESTION -->

### 3.4 Jointly optimizing `max_features` of `CountVectorizer` and `C` of `LogisticRegression` with `RandomizedSearchCV`
rubric={points}

**Your tasks:**

1. Jointly optimize `C` hyperparameter for the `LogisticRegression`, as well as the `max_features` hyperparameter for the `CountVectorizer` using [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Choose reasonable ranges or distributions for the hyperparameters, using either a parameter grid or appropriate distributions at your discretion.
2. Display the best hyperparameter values and the best score found. 

<div class="alert alert-warning">

Solution_3.4
    
</div>

_Points:_ 5

In [ ]:
param_distributions = {
    "countvectorizer__max_features": [10, 100, 1000, 10_000, 100_000],
    "logisticregression__C": 10.0 ** np.arange(-2.0, 2.5, 0.5),
}

rand_search = RandomizedSearchCV(
    pipe_lr,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    random_state=123,
    n_jobs=-1,
)
rand_search.fit(X_train, y_train)

rand_search.best_params_, rand_search.best_score_

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.5 Discussion 
rubric={points}

- Do the best hyperparameter values found through grid search and random search match each other and the results you obtained in 3.1 and 3.2?
- More generally, should these values be expected to match the ones you found in parts 3.1 and 3.2? Why or why not?

<div class="alert alert-warning">

Solution_3.5
    
</div>

_Points:_ 2

Grid search and random search don’t have to return the same best hyperparameters. Random search only samples a subset of combinations, and small CV noise can change which setting looks “best,” especially when multiple settings are very close.

<!-- END QUESTION -->

<br><br>

### 3.6 Test score
rubric={autograde}

**Your tasks:**
- Evaluate your final model on the test set. Store the test accuracy in the variable called `test_score`.

<div class="alert alert-warning">

Solution_3.6
    
</div>

_Points:_ 2

In [ ]:
final_model = grid_search.best_estimator_
test_score = final_model.score(X_test, y_test)
test_score

In [ ]:
grader.check("q3.6")

<br><br>

<!-- BEGIN QUESTION -->

### 3.7 Discussion
rubric={points}

- How does your test accuracy compare to your validation accuracy? 
- If they are different: do you think this is because you "overfitted on the validation set", or simply random luck?

<div class="alert alert-warning">

Solution_3.7
    
</div>

_Points:_ 2

It’s normal for test accuracy to be a bit different from the best CV validation score because the test set is a new sample. If test is noticeably lower, that can mean mild overfitting to the CV process (tuning to validation folds) or simply that the test split is a bit harder.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 4: Very short answer questions
<hr>
rubric={points}

Each question is worth 2 points. Max 2 sentences per answer.

1. What is the problem with calling `fit_transform` on your test data with `CountVectorizer`? 
2. If you could only access one of `predict` or `predict_proba`, which one would you choose? Briefly explain.
3. What are two advantages of `RandomizedSearchCV` over `GridSearchCV`?

<div class="alert alert-warning">

Solution_4
    
</div>

_Points:_ 6

1. Calling fit_transform on the test set with CountVectorizer leaks information from the test data into the fitted vocabulary/feature mapping, so your evaluation becomes optimistically biased.

2. I would choose predict_proba because it contains more information (confidence/probabilities), and you can still recover predict by thresholding those probabilities.

3. (i) RandomizedSearchCV can find good settings with far fewer evaluations when the grid is large, so it’s often faster for the same compute budget. (ii) It can explore a wider range of values (especially for continuous/log-scaled hyperparameters) instead of being restricted to a fixed grid.


<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Here is a quick checklist before submitting: 

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W2/docs/homework-instructions).  

Congratulations on finishing the homework! 

![](./img/eva-well-done.png)